In [2]:
import numpy as np

class AntColony:
    def __init__(self, distance_matrix, n_ants, alpha, beta, evaporation_rate, iterations):
        self.distance_matrix = distance_matrix
        self.n_ants = n_ants
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.iterations = iterations
        self.n_cities = len(distance_matrix)
        self.pheromone_matrix = np.ones_like(distance_matrix) / self.n_cities
        self.best_path = None
        self.best_distance = float('inf')

    def run(self):
        for _ in range(self.iterations):
            paths = self._generate_paths()
            self._update_pheromones(paths)
            self._update_best_path(paths)

    def _generate_paths(self):
        paths = []
        for _ in range(self.n_ants):
            path = [0]  # Start from city 0 (City A)
            while len(path) < self.n_cities:
                allowed_cities = [city for city in range(self.n_cities) if city not in path]
                next_city = self._select_next_city(path, allowed_cities)
                path.append(next_city)
            path.append(0)  # Return to city 0 (City A) to complete the tour
            paths.append(path)
        return paths

    def _select_next_city(self, path, allowed_cities):
        pheromone_values = self.pheromone_matrix[path[-1], allowed_cities]
        attractiveness_values = 1 / self.distance_matrix[path[-1], allowed_cities]
        probabilities = (pheromone_values ** self.alpha) * (attractiveness_values ** self.beta)
        probabilities /= np.sum(probabilities)
        return np.random.choice(allowed_cities, p=probabilities)

    def _update_pheromones(self, paths):
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for path in paths:
            path_distance = sum(self.distance_matrix[path[i], path[i+1]] for i in range(len(path) - 1))
            for i in range(len(path) - 1):
                self.pheromone_matrix[path[i], path[i+1]] += 1 / path_distance

    def _update_best_path(self, paths):
        for path in paths:
            path_distance = sum(self.distance_matrix[path[i], path[i+1]] for i in range(len(path) - 1))
            if path_distance < self.best_distance:
                self.best_path = path
                self.best_distance = path_distance

if __name__ == "__main__":
    # Define distance matrix (example distances)
    distance_matrix = np.array([
        [0, 10, 15, 20, 25],  # Distance from City A to other cities
        [10, 0, 35, 25, 30],  # Distance from City B to other cities
        [15, 35, 0, 30, 20],  # Distance from City C to other cities
        [20, 25, 30, 0, 40],  # Distance from City D to other cities
        [25, 30, 20, 40, 0]   # Distance from City E to other cities
    ])

    # Parameters
    n_ants = 10
    alpha = 1
    beta = 1
    evaporation_rate = 0.1
    iterations = 100

    # Run ACO
    aco = AntColony(distance_matrix, n_ants, alpha, beta, evaporation_rate, iterations)
    aco.run()

    # Output
    print("Best path found:", aco.best_path)
    print("Distance of best path:", aco.best_distance)


Best path found: [0, 2, 4, 1, 3, 0]
Distance of best path: 110
